# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# очень часто сайты могут ограничивать частые запросы к себе,поэтому нужно задерживать исполнение
import time

In [3]:
URL = 'https://habr.com/ru/all'
req = requests.get(URL)
#req.text

In [4]:
soup = BeautifulSoup(req.text, 'html.parser')
#soup

In [5]:
# извлекаем посты
articles = soup.find_all('article', class_='post')
#articles

In [6]:
KEYWORDS = ['oled', 'usb']

In [7]:
list_1 = []

for article in articles:
    posts_text = article.find_all('div', class_='post__text post__text-html post__text_v1') #выделение превью к статьям
    for post_text in posts_text:
        post_text_lower = post_text.text.lower()
        #print(post_text_lower)
        # ищем вхождение хотя бы одного желаемого хаба
        if any([words in post_text_lower for words in KEYWORDS]):
            #print(post_text_lower)
        #else:
            #print("нет")
            # пост нам интересен - делаем с ним все что захотим: можно отправит в телеграм уведомление, можно на почту и т.п.
            title_element = article.find('a', class_='post__title_link')
            a = title_element.attrs.get('href')
            soup = BeautifulSoup(requests.get(a).text, 'html.parser')
            time.sleep(0.3)
            title_element2 = article.find('a', class_='post__title_link').text
            date = article.find('span', class_='post__time').text
            row = {'date': date, 'title': title_element2, 'ссылка': a}
            list_1.append(row)
            # так как пост уже нам подошел - дальше нет смысла проверять хабы
            break

In [8]:
df1 = pd.DataFrame(list_1)
df1

,date,title,ссылка
0,вчера в 23:47,Подключение OLED дисплея ssd1306 к STM32 (SPI+...,https://habr.com/ru/post/514382/
1,вчера в 20:58,Издеваемся над USB,https://habr.com/ru/post/514376/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [9]:
list_ = []

for article in articles:
    posts_text = article.find_all('div', class_='post__text post__text-html post__text_v1') #выделение превью к статьям
    for post_text in posts_text:
        post_text_lower = post_text.text.lower()
        #print(post_text_lower)
        # ищем вхождение хотя бы одного желаемого хаба
        if any([words in post_text_lower for words in KEYWORDS]):
            #print(post_text_lower)
        #else:
            #print("нет")
            # пост нам интересен - делаем с ним все что захотим: можно отправит в телеграм уведомление, можно на почту и т.п.
            title_element = article.find('a', class_='post__title_link')
            a = title_element.attrs.get('href')
            soup = BeautifulSoup(requests.get(a).text, 'html.parser')
            time.sleep(0.3)
            text = soup.find('div', class_='post__text post__text-html post__text_v1').text
            title_element2 = article.find('a', class_='post__title_link').text
            date = article.find('span', class_='post__time').text
            row = {'date': date, 'title': title_element2, 'ссылка': a, 'text': text}
            list_.append(row)
            # так как пост уже нам подошел - дальше нет смысла проверять хабы
            break

In [10]:
df2 = pd.DataFrame(list_)
df2

,date,title,ссылка,text
0,вчера в 23:47,Подключение OLED дисплея ssd1306 к STM32 (SPI+...,https://habr.com/ru/post/514382/,В данной статье будет описан процесс подключен...
1,вчера в 20:58,Издеваемся над USB,https://habr.com/ru/post/514376/,\r\nВ очередной раз втыкая скоростную USB флеш...


In [11]:
text #действительно статья целиком, а не превью

'\r\nВ очередной раз втыкая скоростную USB флешку в порт USB 3.0, я увидел надпись "Это устройство может работать быстрее...". Но подождите, я и так его воткнул в порт 3.0! Неужели контакт барахлит? И если так, то как флешка определяет, на какой скорости ей работать? Ведь современные ПК поддерживают целых три стандарта соединения — USB 1.1, 2.0 и 3.0. Можно ли «понизить» стандарт USB, насильно заставив устройство работать, к примеру, на USB 1.1? Не на все эти вопросы в сети удалось найти ответ, и я решил разобраться сам, по ходу столкнувшись с довольно неочевидными ситуациями.\n\nПонижаем USB 3.0 до 2.0Вы сейчас скажете — да что может быть проще, просто возьми USB 2.0 кабель — и будете совершенно правы. Если в кабеле или разъёме нет USB 3.0 контактов, у устройства не будет иного выхода, кроме как завестись на скорости USB 2.0:\n\n\r\nНо отключение каких конкретно проводков приведёт к переключению на USB 2.0? Что будет, если отключить только один, или замкнуть соседние? Интересно же вый

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [12]:
import json

In [13]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
URL3 = 'https://digibody.avast.com/v1/web/leaks'

In [14]:
email = 'email'
df4a = pd.DataFrame()

for email_el in EMAIL:
    params = {'email': email_el}
    res3 = requests.post(URL3, json=params)

    dict_all = json.loads(res3.text)
    list_row = []
    dict_value = dict_all['value']
    for el in dict_value:
        el = el['leak_info']
        row = {'date': el['date'], 'title': el['title'], 'description': el['description'], email: email_el}
        list_row.append(row) 
        #print(list_info)
    df4 = pd.DataFrame(list_row)
    df4[['date', 'title', 'description', 'email']]
    df4['date'] = pd.to_datetime(df4['date'], unit='ms')
    #print(df4)
    df4a = df4a.append(df4)
df4a

,date,title,description,email
0,2017-12-22,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...,xxx@x.ru
1,2019-02-06,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix...",xxx@x.ru
2,2017-01-31,CD Projekt Red,"In March 2016, CDProjektRed.com.com's forum da...",xxx@x.ru
3,2017-02-14,Parapa,"In July and August 2016, two criminals execute...",xxx@x.ru
4,2017-03-08,Sensitive Source,This source has been marked as sensitive due t...,xxx@x.ru
5,2019-02-06,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix...",xxx@x.ru
6,2019-02-06,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix...",xxx@x.ru
7,2017-02-14,CFire Mail,"In July and August of 2016, two criminals carr...",xxx@x.ru
8,2016-10-21,Adobe Systems,"In October of 2013, criminals penetrated Adobe...",xxx@x.ru
9,2019-01-29,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix...",xxx@x.ru


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
